In [1]:
import pandas as pd
import re
import warnings

In [2]:
warnings.filterwarnings('ignore')

I valmyndighetens kandidatsdata finns en kolumn som heter "valsedelsuppgift". I den så brukar kandidaterna skriva sitt yrke. Det är denna uppgift som jag här under försöker plocka fram.

I den här processen har textuppgifterna från kandidaternas valsedalr tvättats upprepade gånger i [Open Refine](http://openrefine.org/), ett verktyg jag starkt rekommenderar för den som vill metodologiskt tvätta textdata.

# Kommunerna

In [98]:
df = pd.read_csv('data/kandidaturer.skv',sep=';', encoding = "ISO-8859-1")

In [99]:
df.shape

(184197, 24)

In [100]:
df.columns = df.columns.str.lower()

In [101]:
partier = ['M','L','C','KD','S','V','MP','SD']

In [102]:
df = df.loc[df.partiförkortning.isin(partier)]

In [103]:
df.columns

Index(['valtyp', 'valområdeskod', 'valområdesnamn', 'valkretskod',
       'valkretsnamn', 'partibeteckning', 'partiförkortning', 'partikod',
       'valsedelsstatus', 'listnummer', 'ordning', 'anmkand', 'anmdeltagande',
       'samtycke', 'förklaring', 'kandidatnummer', 'namn', 'ålder_på_valdagen',
       'kön', 'folkbokföringsort', 'valsedelsuppgift', 'ant_best_vals',
       'valbar_på_valdagen', 'giltig'],
      dtype='object')

In [104]:
df['valsedelsuppgift'].isnull().value_counts()

False    145973
True       4957
Name: valsedelsuppgift, dtype: int64

In [105]:
df = df[['valtyp','namn','partiförkortning','kandidatnummer','valsedelsuppgift']].rename(columns={'partiförkortning':'parti'})

In [106]:
df.shape

(150930, 5)

In [107]:
df.dropna().shape # bort med alla som saknar valsedelsuppgift

(145973, 5)

In [108]:
df = df.dropna()

In [109]:
def number_ignorer(value):
    r = re.match(".*\d+.*",value)
    if not r:
        return value
    else: 
        return

In [110]:
df['tmp'] = df.valsedelsuppgift.str.split(',').apply(lambda x: [number_ignorer(y).strip() for y in x if number_ignorer(y)])

In [111]:
df['tmp_len'] = df.tmp.apply(lambda x: len(x))

In [112]:
df.tmp_len.value_counts()

2    100835
1     40264
0      3505
3      1324
4        39
5         6
Name: tmp_len, dtype: int64

In [113]:
df=df.reset_index(drop=True)

In [114]:
övriga = df.loc[df['valtyp']!='K']

In [115]:
df = df.loc[df['valtyp']=='K']

In [116]:
df = df.set_index('kandidatnummer')

Den här _jävligt_ feta lösningen på att få ut alla listvärdena i egna kolumner [kommer från detta svar](https://stackoverflow.com/questions/35491274/pandas-split-column-of-lists-into-multiple-columns):

In [117]:
tvätt = pd.DataFrame(df.tmp.values.tolist(),index=df.index)

In [118]:
tvätt.columns = ['col1','col2','col3','col4']

In [119]:
for col in tvätt.columns:
    tvätt[col] = tvätt[col].str.lower()

In [120]:
tvätt = tvätt.reset_index().melt(id_vars='kandidatnummer')

In [121]:
tvätt['value'] = tvätt.value.str.lower().str.strip()

In [122]:
tvätt = tvätt.dropna()

In [123]:
tvätt_data = tvätt.value.value_counts().loc[tvätt.value.value_counts()>2].reset_index().rename(columns={'index':'grund'})

In [124]:
tvätt_data['tvätt'] = tvätt_data['grund']

In [125]:
tvätt_data.drop('value',axis=1).to_csv('data/yrken_att_tvättas.csv',index=False)

In [126]:
df1 = pd.read_csv('data/yrken_tvättade1.csv')

In [127]:
df1.shape

(3603, 3)

In [128]:
df1.loc[df1['tvätt2']!='skräp'].to_csv('data/yrken_tvättade1.csv',index=False)

Ovan fil har sedan tvättats återigen i refine till följande resultat:

In [129]:
tvättad_data = pd.read_csv('data/yrken_tvättade2.csv')[['grund','tvätt3']]

In [130]:
data = tvätt.rename(columns={'value':'grund'})

In [131]:
data = data.merge(tvättad_data,on='grund',how='outer')[['kandidatnummer','tvätt3']]

In [132]:
data = data.loc[~data['tvätt3'].isnull()]

In [133]:
data = data.drop_duplicates()

In [134]:
data = data.merge(data.kandidatnummer.value_counts().reset_index().rename(columns={'index':'kandidatnummer','kandidatnummer':'antal'}),on='kandidatnummer',how='left')

All data är tvättad och ser korrekt ut, men det verkar som att det finns en del som har uppgivit mer än ett yrke. Min lösning blir därför att låta dessa dubletter vara kvar. Så en politiker kan räknas två gånger ifall hen har två yrken. Detta är dock inget större problem då dessa motsvarar ca 1,5 procent av samtliga. T ex "civilekonom" och "civilekonom". Se nedan ihopslagning för att se hur det kan se ut:

In [135]:
data.loc[data['antal']>1].sort_values('kandidatnummer').merge(df.reset_index()[['kandidatnummer','valsedelsuppgift']],on='kandidatnummer',how='left').head(10)

,kandidatnummer,tvätt3,antal,valsedelsuppgift
0,1986,ekonom,2,"ekonom, pensionär, Bokenäs"
1,1986,pensionär,2,"ekonom, pensionär, Bokenäs"
2,12411,fil. dr.,2,"71 år, Fil.Dr, Universitetslärare"
3,12411,fil. dr.,2,"71 år, Fil.Dr, Universitetslärare"
4,12411,fil. dr.,2,"71 år, Fil.Dr, Universitetslärare"
5,12411,universitetslärare,2,"71 år, Fil.Dr, Universitetslärare"
6,12411,universitetslärare,2,"71 år, Fil.Dr, Universitetslärare"
7,12411,universitetslärare,2,"71 år, Fil.Dr, Universitetslärare"
8,12412,fil. mag.,2,"29 år, Fil.Mag, Statstjänsteman"
9,12412,fil. mag.,2,"29 år, Fil.Mag, Statstjänsteman"


In [136]:
data.shape

(30670, 3)

In [137]:
data = data.merge(df.reset_index()[['kandidatnummer','valtyp','valsedelsuppgift']],on='kandidatnummer',how='left')

In [138]:
data.loc[data['antal']>1].shape

(779, 5)

In [139]:
data = data.rename(columns={'tvätt3':'tvätt'})[['kandidatnummer','tvätt','valtyp','valsedelsuppgift']]

In [140]:
test = df.reset_index()

In [141]:
test = test.merge(test.kandidatnummer.value_counts().reset_index().rename(columns={'index':'kandidatnummer','kandidatnummer':'antal'}),on='kandidatnummer',how='left')

In [142]:
test.head()

,kandidatnummer,valtyp,namn,parti,valsedelsuppgift,tmp,tmp_len,antal
0,202001,K,Mathias Bohman,S,"44, Kommunstyrelsens ordförande, Centrala Väsby","[Kommunstyrelsens ordförande, Centrala Väsby]",2,1
1,202002,K,Suzanne Granqvist Enell,S,"53, Försäljningschef, Bollstanäs","[Försäljningschef, Bollstanäs]",2,1
2,202003,K,Ibrahim Abdallah,S,"39, Student, Grimsta","[Student, Grimsta]",2,1
3,202004,K,Barbro Brolin,S,"61, Autismpedagog, Runby","[Autismpedagog, Runby]",2,1
4,202005,K,Gert Lindström,S,"62, Byggnadsarbetare, Centrala Väsby","[Byggnadsarbetare, Centrala Väsby]",2,1


In [143]:
test = test.drop(['tmp','antal'],axis=1).drop_duplicates()

In [144]:
test = test.merge(test.kandidatnummer.value_counts().reset_index().rename(columns={'index':'kandidatnummer','kandidatnummer':'antal'}),on='kandidatnummer',how='left')

In [145]:
test.loc[test.antal>1].sort_values('kandidatnummer').head()

,kandidatnummer,valtyp,namn,parti,valsedelsuppgift,tmp_len,antal
25984,1895,K,Erik Norderby,C,"34 år,Nordby,Fjällbacka",2,2
25992,1895,K,Erik Norderby,C,"Nordby, Fjällbacka",2,2
25961,11734,K,Thomas Fransson,C,"Villavägen, Bullaren",2,2
25983,11734,K,Thomas Fransson,C,"30 år,Villavägen,Bullaren",2,2
3778,13860,K,Anita Lundin,M,"71, Fil. mag",1,2


In [51]:
test.loc[test['antal']>1].sort_values('kandidatnummer').parti.value_counts()

SD    311
S     128
MP     69
KD     14
M      14
V       4
C       4
Name: parti, dtype: int64

Ovan kan vi se de kandidater som finns på två olika valsedlar (som är skrivna olika). Av dessa är mer än hälften kandidater från SD. Vi ignorerar dessa.

Nu ska vi plocka fram alla som är invalda!

In [149]:
data.rename(columns={'kandidatnummer':'id'}).to_csv('data/kommunpol_yrken.csv',index=False)

# Riket och landstingen

Jag är ett ägg och sorterade bort alla kandidater för riksdagen och landstingen. Så jag behövde tvätta en gång till >:(

Ovanför är då alltså alla kommunkandidater. Nu för alla övriga:

In [67]:
övriga = övriga.set_index('kandidatnummer') # filen kandidaturer.skv som tvättats och innehållandes endast R och L

In [68]:
tvätt = pd.DataFrame(övriga.tmp.values.tolist(),index=övriga.index)

In [69]:
tvätt.columns = ['col1','col2','col3','col4','col5']

In [70]:
for col in tvätt.columns:
    tvätt[col] = tvätt[col].str.lower()

In [71]:
tvätt = tvätt.reset_index().melt(id_vars='kandidatnummer')

In [72]:
tvätt['value'] = tvätt.value.str.lower().str.strip()

In [73]:
tvätt = tvätt.dropna()

In [74]:
tvätt_data = tvätt.value.value_counts().loc[tvätt.value.value_counts()>2].reset_index().rename(columns={'index':'grund'})

In [75]:
tvätt_data['tvätt'] = tvätt_data['grund']

In [76]:
test_rens = pd.read_csv('data/yrken_tvättade2.csv')[['grund','tvätt3']]

In [77]:
test = test_rens.merge(tvätt_data,on='grund',how='left')[['grund','tvätt','tvätt3']].dropna()

In [78]:
tvätt_data = tvätt_data[['grund','tvätt']]

In [79]:
tvätt_data = tvätt_data.merge(test.rename(columns={'tvätt':'tvätt_K'}),on='grund',how='left')

In [80]:
tvätt_data.loc[~tvätt_data['tvätt3'].isnull(),'tvätt'] = \
tvätt_data.loc[~tvätt_data['tvätt3'].isnull(),'tvätt3']

Så denna data ska nu tvättas i Refine:

In [81]:
tvätt_data[['grund','tvätt']].to_csv('data/tvtt_övriga.csv',index=False)

Och detta är resultatet av det:

In [82]:
df1 = pd.read_csv('data/tvätt_övriga2.csv')

In [83]:
data = tvätt.rename(columns={'value':'grund'})

In [84]:
data = data.merge(df1,on='grund',how='outer')[['kandidatnummer','grund','tvätt']]

In [85]:
data = data.loc[data['tvätt']!='ort']

In [86]:
data = data.loc[~data['tvätt'].isnull()]

In [87]:
data = data.drop_duplicates()

In [88]:
data = data.merge(data.kandidatnummer.value_counts().reset_index().rename(columns={'index':'kandidatnummer','kandidatnummer':'antal'}),on='kandidatnummer',how='left')

In [89]:
riks_län = data.groupby(['kandidatnummer','grund','tvätt']).count().reset_index()

In [90]:
riks_län.head()

,kandidatnummer,grund,tvätt,antal
0,1,projektledare,projektledare,1
1,3,handläggare,handläggare,1
2,4,enhetschef,chef,1
3,5,handläggare,handläggare,1
4,8,socialsekreterare,socialarbetare,1


In [91]:
riks_län = riks_län.merge(övriga.reset_index()[['kandidatnummer',
                                                'valtyp',
                                                'valsedelsuppgift']],
                          on='kandidatnummer',
                          how='left')\
                    .drop_duplicates()\
                    .drop(['grund',
                           'antal'],
                          axis=1).rename(columns={'kandidatnummer':'id'})

In [150]:
riks_län.head()

,id,tvätt,valtyp,valsedelsuppgift
0,1,projektledare,R,"Projektledare, Sala"
1,1,projektledare,L,"Projektledare, Sala"
3,3,handläggare,R,"Handläggare, Sala"
4,3,handläggare,L,"Handläggare, Sala"
6,4,chef,R,"Enhetschef, Sala"


In [87]:
riks_län.to_csv('data/rikslänpol_yrken.csv',index=False)

# Yrken samtliga kandidater

In [155]:
samtliga = pd.concat([pd.read_csv('data/kommunpol_yrken.csv'),pd.read_csv('data/rikslänpol_yrken.csv')])

In [158]:
samtliga = samtliga[['id','tvätt','valsedelsuppgift']].drop_duplicates()

In [159]:
samtliga.to_csv('data/yrken_alla_kand.csv',index=False)